# Описание проекта

Необходимо проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

# Загрузка и обзор данных

In [ ]:
import pandas as pd # импорт библиотеки pandas

In [ ]:
df = pd.read_csv('/datasets/yandex_music_project.csv') # чтение файла с данными и сохранение в df

In [ ]:
display(df.head(10)) # получение первых 10 строк таблицы df

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [ ]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Слитное написание двух слов вместо "змеиного_регистра".



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

# Предобработка данных

## Стиль заголовков

In [ ]:
df.columns # перечень названий столбцов таблицы df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [ ]:
df = df.rename(columns = {'  userID' : 'user_id', 'Track' : 'track', '  City  ' : 'city', 'Day' : 'day'}) # переименование столбцов

In [ ]:
display(df.columns) # проверка результатов - перечень названий столбцов

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

## Пропуски значений

In [ ]:
display(df.isna().sum()) # подсчёт пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

In [ ]:
columns_to_replace = ['track', 'artist', 'genre']

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')
    
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

In [ ]:
display(df.isna().sum()) # подсчёт пропусков

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

## Дубликаты

In [ ]:
display(df.duplicated().sum()) # подсчёт явных дубликатов

3826

In [ ]:
df = df.drop_duplicates().reset_index(drop = True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)

In [ ]:
display(df.duplicated().sum()) # проверка на отсутствие дубликатов

0

In [ ]:
df = df.sort_values('genre')
display(df['genre'].unique())

# Просмотр уникальных названий жанров

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [ ]:
def replace_wrong_genres(wrong_genres, correct_genre):
    for genre in wrong_genres:
        df['genre'] = df['genre'].replace(genre, correct_genre)
    
    # Функция для замены неявных дубликатов    

In [ ]:
replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop') # Устранение неявных дубликатов

In [ ]:
df = df.sort_values('genre')
display(df['genre'].unique())
# Проверка на неявные дубликаты

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

# Проверка гипотез

## Сравнение поведения пользователей двух столиц

Проверка первой гипотезы: пользователи по-разному слушают музыку в Москве и Санкт-Петербурге.

Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице

In [ ]:
df.groupby('city')['time'].count() # Подсчёт прослушиваний в каждом городе

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64

In [ ]:
df.groupby('day')['city'].count() # Подсчёт прослушиваний в каждый из трёх дней

day
Friday       21840
Monday       21354
Wednesday    18059
Name: city, dtype: int64

In [ ]:
def number_tracks(day, city):
    track_list = df[(df['day'] == day)&(df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count 
# Функция для подсчёта прослушиваний для конкретного города и дня

In [ ]:
number_tracks('Monday', 'Moscow') # количество прослушиваний в Москве по понедельникам

15740

In [ ]:
number_tracks('Monday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [ ]:
number_tracks('Wednesday', 'Moscow') # количество прослушиваний в Москве по средам

11056

In [ ]:
number_tracks('Wednesday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по средам

7003

In [ ]:
number_tracks('Friday', 'Moscow') # количество прослушиваний в Москве по пятницам

15945

In [ ]:
number_tracks('Friday', 'Saint-Petersburg') # количество прослушиваний в Санкт-Петербурге по пятницам

5895

In [ ]:
pd.DataFrame(data = [
    [
        'Moscow', 
        number_tracks('Monday', 'Moscow'), 
        number_tracks('Wednesday', 'Moscow'), 
        number_tracks('Friday', 'Moscow')], 
    [
        'Saint-Petersburg', 
        number_tracks('Monday', 'Saint-Petersburg'), 
        number_tracks('Wednesday', 'Saint-Petersburg'), 
        number_tracks('Friday', 'Saint-Petersburg')]
], columns = [
    'city',
    'monday', 
    'wednesday', 
    'friday']) # Таблица с результатами

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

## Музыка в начале и в конце недели

Проверим вторую гипотезу: утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [ ]:
moscow_general = df[df['city'] == 'Moscow'] # получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'

In [ ]:
spb_general = df[df['city'] == 'Saint-Petersburg'] # получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'

In [ ]:
def genre_weekday(table, day, time1, time2): # Объявление функции genre_weekday() с параметрами table, day, time1, time2,
    genre_df = table[(table['day'] == day) & (table['time'] > time1) & (table['time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return genre_df_sorted.head(10)

# функция возвращает информацию о самых популярных жанрах в указанный день в заданное время

In [ ]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00') # вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)

genre
pop            781
dance          549
electronic     480
rock           474
hip            281
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [ ]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00') # вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)

genre
pop            218
dance          182
rock           162
electronic     147
hip             79
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [ ]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00') # вызов функции для вечера пятницы в Москве

genre
pop            713
rock           517
dance          495
electronic     482
hip            267
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [ ]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00') # вызов функции для вечера пятницы в Петербурге

genre
pop            256
electronic     216
rock           216
dance          210
hip             94
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

## Жанровые предпочтения в Москве и Петербурге

Проверим третью гипотезу: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве, а в Москве преобладает поп-музыка.

In [ ]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)

In [ ]:
display(moscow_genres.head(10)) # просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hip            2041
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [ ]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending = False)

In [ ]:
display(spb_genres.head(10)) # просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hip             934
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


# Итоги исследования

Гипотезы проверены и установлено следующее:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.